In [113]:
import numpy as np
import pandas as pd

In [114]:
df = pd.read_csv("./../data.csv", names=["x1", "x2"])
df

,x1,x2
0,-0.0824,0.9435
1,0.0913,0.9575
2,0.0476,1.0683
3,0.2311,1.0452
4,0.2269,0.9615
...,...,...
96,-0.5031,1.8191
97,-0.3340,1.8515
98,-0.3200,1.9158
99,-0.2074,1.9929


In [115]:
def calculate_all_distances_sq(probki, V):
    m = V.shape[0]
    D = np.zeros((m, probki.shape[0]))
    for j in range(m):
        for s in range(probki.shape[0]):
            D[j, s] = np.sum((probki[s, :] - V[j, :]) ** 2)
    return D

In [116]:
def fuzzy_c_means(probki, m, fcm_m, max_iters, epsilon=1e-5):
    M, n = probki.shape
    V_history = []

    # 1.1 Stworzenie tablic U i D o rozmiarze m x M, gdzie m to liczba klas, a M to liczba próbek(wartość dowolna)
    # 1.3 Wypełnienie tablicy D losowymi wartościami dodatnimi, na przykład z przedziału (0,1; 1,1).
    D = np.random.uniform(0.1, 1.1, size=(m, M))
    exp_factor = 1.0 / (1.0 - fcm_m)
    # 1.4 Wyliczenie każdej wartości w tablicy Uj,s (j=1..m; s=1..M).
    U = (D ** exp_factor) / np.sum(D ** exp_factor, axis=0)

    # 1.2 Utworzenie tablicy V ze środkami grup o rozmiarze m x n, gdzie m to liczba grup, a n to liczba atrybutów (wartość dowolna).
    U_m = U ** fcm_m
    V = (U_m @ probki) / np.sum(U_m, axis=1, keepdims=True)

    V_history.append((V.copy(), U.copy()))

    # 2. Główna pętla programu wykonywana przez zadaną liczbę iteracji.
    for iter_count in range(1, max_iters + 1):
        V_old = V.copy()

        # 2.1. Obliczenie odległości między każdą próbką a grupą: Dj,s (j=1..m; s=1..M, m to liczba grup, M to liczba próbek)
        D = calculate_all_distances_sq(probki, V)

        # 2.2. Należy zadbać, aby wszystkie wartości w tablicy D były większe od ustalonej małej wartości (na przykład wszystkie wartości < 1e-5 zastąpić 1e-5).
        D[D < epsilon] = epsilon

        # 2.3. Wyliczenie stopnia przynależności poszczególnej próbki do każdej grupy: Uj,s (j=1..m; s=1..M, m to liczba grup, M to liczba próbek).
        inv_D_exp = (1.0 / D) ** (1.0 / (fcm_m - 1.0))
        U = inv_D_exp / np.sum(inv_D_exp, axis=0, keepdims=True)

        # 2.4. Należy sprawdzić, czy przypadkiem U nie zawiera wartości nieoznaczonych (w takim przypadku należy przerwać program i wyświetlić komunikat ostrzegawczy)
        if np.any(np.isnan(U)):
            print(f"U zawiera wartości nieoznaczone")
            break

        # 2.5. Obliczenie nowych położeń środków grup Vj,i (j=1..m; i=1..n; m to liczba grup, n to liczba atrybutów)
        U_m = U ** fcm_m
        V = (U_m @ probki) / np.sum(U_m, axis=1, keepdims=True)

        V_history.append((V.copy(), U.copy()))

        if np.max(np.abs(V - V_old)) < 1e-4:
            break

    return U, V, V_history

In [117]:
probki = df.values
M, n = probki.shape
m = 4
fcm_m = 2
max_iters = 20

U_final, V_final, V_history = fuzzy_c_means(probki, m, fcm_m, max_iters)

In [118]:
def save_raport(V_history, iteration, threshold=0.6):
    V, U = V_history[iteration]
    m = V.shape[0]
    rows = []

    for j in range(m):
        mask = U[j, :] > threshold
        grupa_j = probki[mask, :]
        liczba_probek = len(grupa_j)

        if liczba_probek > 0:
            min_x1 = np.min(grupa_j[:, 0])
            max_x1 = np.max(grupa_j[:, 0])
            min_x2 = np.min(grupa_j[:, 1])
            max_x2 = np.max(grupa_j[:, 1])
        else:
            min_x1, max_x1, min_x2, max_x2 = (np.nan, np.nan, np.nan, np.nan)

        center = f"({V[j, 0]}, {V[j, 1]})"

        rows.append({
            'Grupa': j + 1,
            'Położenie środka': center,
            'Liczba próbek': liczba_probek,
            'Min_x1': min_x1,
            'Max_x1': max_x1,
            'Min_x2': min_x2,
            'Max_x2': max_x2
        })
    raport_df = pd.DataFrame(rows)
    raport_df.to_excel(f"Fuzzy c-means{iteration}.xlsx", index=False)
    print(raport_df)


In [119]:
save_raport(V_history, 4)
save_raport(V_history, 20)

   Grupa                            Położenie środka  Liczba próbek  Min_x1  \
0      1   (-1.3562337733682692, 0.9735125462673976)             19 -1.7787   
1      2  (-0.6343150250411912, -1.2423603040590805)             19 -1.3595   
2      3    (0.3961773462146656, 0.9002229592635448)             16 -0.0824   
3      4    (1.056938482131681, -0.3835487932602895)             20  0.8467   

   Max_x1  Min_x2  Max_x2  
0 -0.8084  0.1831  1.8031  
1  0.1774 -1.5564 -0.8821  
2  0.8566  0.6752  1.0724  
3  1.3375 -1.0805  0.3286  
   Grupa                           Położenie środka  Liczba próbek  Min_x1  \
0      1   (-1.2902656304087992, 1.134081707984291)             18 -1.7544   
1      2  (-1.0367272761375919, -1.058376507546722)             18 -1.5488   
2      3    (0.624693319334989, 0.7976534809210407)             22 -0.0824   
3      4  (0.8635506967753758, -0.8725609755733168)             20  0.2684   

   Max_x1  Min_x2  Max_x2  
0 -0.6135  0.2806  1.8584  
1 -0.3374 -1.5191